In [1]:
FN = 'train'
FN0 = 'vocabulary-embedding'
FN1 = 'train'

import os
import keras
keras.__version__


Using TensorFlow backend.


'2.1.3'

In [2]:
maxlend=25 # 0 - if we dont want to use description at all
maxlenh=25
maxlen = maxlend + maxlenh
rnn_size = 512 # must be same as 160330-word-gen
rnn_layers = 3  # match FN1
batch_norm=False

In [3]:
activation_rnn_size = 40 if maxlend else 0
#activation_rnn_size = 0

In [4]:
# training parameters
seed=42
p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size=20
nflips=10

In [5]:
# nb_train_samples = 30000
# nb_val_samples = 3000
nb_train_samples = 250
nb_val_samples = 50

In [6]:
import pickle

with open('%s.pkl'%FN0, 'rb') as fp:
    embedding, idx2word, word2idx, glove_idx2idx = pickle.load(fp)
vocab_size, embedding_size = embedding.shape

In [7]:
with open('%s.data.pkl'%FN0, 'rb') as fp:
    X, Y = pickle.load(fp)

In [8]:
nb_unknown_words = 0

In [9]:
print ('number of examples',len(X),len(Y))
print ('dimension of embedding space for words',embedding_size)
print ('vocabulary size', vocab_size, 'the last %d words can be used as place holders for unknown/oov words'%nb_unknown_words)
print ('total number of different words',len(idx2word), len(word2idx))
print ('number of words outside vocabulary which we can substitue using glove similarity', len(glove_idx2idx))
print ('number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov)',len(idx2word)-vocab_size-len(glove_idx2idx))

number of examples 379 379
dimension of embedding space for words 100
vocabulary size 40000 the last 0 words can be used as place holders for unknown/oov words
total number of different words 9075 9075
number of words outside vocabulary which we can substitue using glove similarity 0
number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov) -30925


In [10]:
print(len(idx2word)-vocab_size-len(glove_idx2idx))
print(len(idx2word))
print(vocab_size)
print(len(glove_idx2idx))

-30925
9075
40000
0


In [11]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i

In [12]:
oov0 = vocab_size-nb_unknown_words

In [13]:
for i in range(oov0, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [14]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=nb_val_samples, random_state=seed)
len(X_train), len(Y_train), len(X_test), len(Y_test)

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(329, 329, 50, 50)

In [15]:
del X
del Y

In [16]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [17]:
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
import random, sys

In [18]:
def prt(label, x):
    print (label+':',)
    for w in x:
        print (idx2word[w],)
    print


In [19]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.layers import merge, SpatialDropout1D
from keras.callbacks import TensorBoard

In [20]:
# seed weight initialization
random.seed(seed)
np.random.seed(seed)

In [21]:
regularizer = l2(weight_decay) if weight_decay else None

In [22]:
vocab_size, embedding_size

(40000, 100)

NEW: Testing MODEL

In [51]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size,
                    input_length=maxlen,
                    W_regularizer=regularizer, weights=[embedding], mask_zero=True,
                    name='embedding_1'))
model.add(SpatialDropout1D(p_emb))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True,
                W_regularizer=regularizer, U_regularizer=regularizer,
                b_regularizer=regularizer, dropout_W=0, dropout_U=0,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)
    model.add(Dropout(p_dense,name='dropout_%d'%(i+1)))

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(40000, 100, input_length=50, weights=[array([[-..., mask_zero=True, name="embedding_1", embeddings_regularizer=None)`
  """
/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, name="lstm_1", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  # This is added back by InteractiveShellApp.init_path()
/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, name="lstm_2", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  # This is added back by InteractiveShellApp.init_path()
/Users

In [48]:
maxlenh

25

In [46]:
from keras.layers.core import Lambda
import keras.backend as K

def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    print("dim", X.get_shape())
    desc, head = X[:maxlend,:], X[maxlend:,:]
    head_activations, head_words = head[:,:,:n], head[:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    #print(head_activations)
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    # make sure we dont use description words that are masked out
    #print(activation_energies)
    print("made here")
    #print("mask", mask[:, :maxlend])
    activation_energies = activation_energies + -1e20*K.expand_dims(1.-K.cast(mask[:, :maxlend],'float32'),1)
    print("made here2")
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=(2,1))
    return K.concatenate((desc_words, head_words))


class SimpleContext(Lambda):
    def __init__(self,**kwargs):
        super(SimpleContext, self).__init__(simple_context,**kwargs)
        self.supports_masking = True

    def compute_mask(self, input, input_mask=None):
        return input_mask[:, maxlend:]
    
    def get_output_shape_for(self, input_shape):
        nb_samples = input_shape[0]
        n= 2 * rnn_size
        #n = 2*(rnn_size - activation_rnn_size)
        return (nb_samples, maxlenh, n)

In [47]:
if activation_rnn_size:
    model.add(SimpleContext(name='simplecontext_1'))
# model.add(TimeDistributed(Dense(vocab_size,
#                                 W_regularizer=regularizer, b_regularizer=regularizer,
#                                 name = 'timedistributed_1')))
# model.add(Activation('softmax', name='activation_1'))

dim (?, ?, 512)
made here
made here2
dim (?, 50, 512)
made here


TypeError: 'NoneType' object is not subscriptable

In [26]:
from keras.optimizers import Adam, RMSprop # usually I prefer Adam but article used rmsprop
# opt = Adam(lr=LR)  # keep calm and reduce learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [27]:
K.set_value(model.optimizer.lr,np.float32(LR))

In [442]:
def str_shape(x):
    return 'x'.join(map(str,x.shape))
    
def inspect_model(model):
    for i,l in enumerate(model.layers):
        print (i, 'cls=%s name=%s'%(type(l).__name__, l.name))
        weights = l.get_weights()
        for weight in weights:
            print (str_shape(weight))
        

In [458]:
inspect_model(model)

0 cls=Embedding name=embedding_1
40000x100
1 cls=LSTM name=lstm_1
100x2048
512x2048
2048
2 cls=Dropout name=dropout_1
3 cls=LSTM name=lstm_2
512x2048
512x2048
2048
4 cls=Dropout name=dropout_2
5 cls=LSTM name=lstm_3
512x2048
512x2048
2048
6 cls=Dropout name=dropout_3
7 cls=TimeDistributed name=time_distributed_15
512x40000
40000
8 cls=Activation name=activation_1


In [365]:
if FN1 and os.path.exists('data/%s.hdf5'%FN1):
    model.load_weights('data/%s.hdf5'%FN1)

In [366]:
def lpadd(x, maxlend=maxlend, eos=eos):
    """left (pre) pad a description to maxlend and then add eos.
    The eos is the input to predicting the first word in the headline
    """
    assert maxlend >= 0
    if maxlend == 0:
        return [eos]
    n = len(x)
    if n > maxlend:
        x = x[-maxlend:]
        n = maxlend
    return [empty]*(maxlend-n) + x + [eos]

In [367]:
samples = [lpadd([3]*26)]
# pad from right (post) so the first maxlend will be description followed by headline
data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')

In [368]:
np.all(data[:,maxlend] == eos)

True

In [369]:
map(len, samples)

In [370]:
data.shape,map(len, samples)

((1, 50), <map at 0x1a476db6d8>)

In [371]:
probs = model.predict(data, verbose=0, batch_size=1)
probs.shape

(1, 50, 40000)

In [372]:
# variation to https://github.com/ryankiros/skip-thoughts/blob/master/decoding/search.py
def beamsearch(predict, start=[empty]*maxlend + [eos],
               k=1, maxsample=maxlen, use_unk=True, empty=empty, eos=eos, temperature=1.0):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    def sample(energy, n, temperature=temperature):
        """sample at most n elements according to their energy"""
        n = min(n,len(energy))
        prb = np.exp(-np.array(energy) / temperature )
        res = []
        for i in range(n):
            z = np.sum(prb)
            r = np.argmax(np.random.multinomial(1, prb/z, 1))
            res.append(r)
            prb[r] = 0. # make sure we select each element only once
        return res

    dead_k = 0 # samples that reached eos
    dead_samples = []
    dead_scores = []
    live_k = 1 # samples that did not yet reached eos
    live_samples = [list(start)]
    live_scores = [0]

    while live_k:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        #print(len(cand_scores[0]))
        if (len(cand_scores)==1):
            break
        cand_scores[:,empty] = 1e20
        if not use_unk:
            for i in range(nb_unknown_words):
                cand_scores[:,vocab_size - 1 - i] = 1e20
        live_scores = list(cand_scores.flatten())
        

        # find the best (lowest) scores we have from all possible dead samples and
        # all live samples and all possible new words added
        scores = dead_scores + live_scores
        ranks = sample(scores, k)
        n = len(dead_scores)
        ranks_dead = [r for r in ranks if r < n]
        ranks_live = [r - n for r in ranks if r >= n]
        
        dead_scores = [dead_scores[r] for r in ranks_dead]
        dead_samples = [dead_samples[r] for r in ranks_dead]
        
        live_scores = [live_scores[r] for r in ranks_live]

        # append the new words to their appropriate live sample
        voc_size = probs.shape[1]
        live_samples = [live_samples[r//voc_size]+[r%voc_size] for r in ranks_live]

        # live samples that should be dead are...
        # even if len(live_samples) == maxsample we dont want it dead because we want one
        # last prediction out of it to reach a headline of maxlenh
        zombie = [s[-1] == eos or len(s) > maxsample for s in live_samples]
        
        # add zombies to the dead
        dead_samples += [s for s,z in zip(live_samples,zombie) if z]
        dead_scores += [s for s,z in zip(live_scores,zombie) if z]
        dead_k = len(dead_samples)
        # remove zombies from the living 
        live_samples = [s for s,z in zip(live_samples,zombie) if not z]
        live_scores = [s for s,z in zip(live_scores,zombie) if not z]
        live_k = len(live_samples)

    return dead_samples + live_samples, dead_scores + live_scores

In [373]:
def keras_rnn_predict(samples, empty=empty, model=model, maxlen=maxlen):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    sample_lengths = map(len, samples)
    assert all(l > maxlend for l in sample_lengths)
    assert all(l[maxlend] == eos for l in samples)
    # pad from right (post) so the first maxlend will be description followed by headline
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')
    probs = model.predict(data, verbose=0, batch_size=batch_size)
    return np.array([prob[sample_length-maxlend-1] for prob, sample_length in zip(probs, sample_lengths)])

In [374]:
def vocab_fold(xs):
    """convert list of word indexes that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
    """
    xs = [x if x < oov0 else glove_idx2idx.get(x,x) for x in xs]
    # the more popular word is <0> and so on
    outside = sorted([x for x in xs if x >= oov0])
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x,vocab_size-1-min(i, nb_unknown_words-1)) for i, x in enumerate(outside))
    xs = [outside.get(x,x) for x in xs]
    return xs

In [375]:
def vocab_unfold(desc,xs):
    # assume desc is the unfolded version of the start of xs
    unfold = {}
    for i, unfold_idx in enumerate(desc):
        fold_idx = xs[i]
        if fold_idx >= oov0:
            unfold[fold_idx] = unfold_idx
    return [unfold.get(x,x) for x in xs]

In [376]:
import sys
import Levenshtein

def gensamples(skips=2, k=10, batch_size=batch_size, short=True, temperature=1., use_unk=True):
    i = random.randint(0,len(X_test)-1)
    print ('HEAD:',' '.join(idx2word[w] for w in Y_test[i][:maxlenh]))
    print ('DESC:',' '.join(idx2word[w] for w in X_test[i][:maxlend]))
    sys.stdout.flush()

    print ('HEADS:')
    x = X_test[i]
    samples = []
    if maxlend == 0:
        skips = [0]
    else:
        skips = range(min(maxlend,len(x)), max(maxlend,len(x)), abs(maxlend - len(x)) // skips + 1)
    for s in skips:
        start = lpadd(x[:s])
        fold_start = vocab_fold(start)
        sample, score = beamsearch(predict=keras_rnn_predict, start=fold_start, k=k, temperature=temperature, use_unk=use_unk)
        assert all(s[maxlend] == eos for s in sample)
        samples += [(s,start,scr) for s,scr in zip(sample,score)]

    samples.sort(key=lambda x: x[-1])
    codes = []
    for sample, start, score in samples:
        code = ''
        words = []
        sample = vocab_unfold(start, sample)[len(start):]
        for w in sample:
            if w == eos:
                break
            words.append(idx2word[w])
            code += chr(w//(256*256)) + chr((w//256)%256) + chr(w%256)
        if short:
            distance = min([100] + [-Levenshtein.jaro(code,c) for c in codes])
            if distance > -0.6:
                print(score, ' '.join(words))
        #         print '%s (%.2f) %f'%(' '.join(words), score, distance)
        else:
                print (score, ' '.join(words))
        codes.append(code)

In [377]:
def flip_headline(x, nflips=None, model=None, debug=False):
    """given a vectorized input (after `pad_sequences`) flip some of the words in the second half (headline)
    with words predicted by the model
    """
    if nflips is None or model is None or nflips <= 0:
        return x
    
    batch_size = len(x)
    assert np.all(x[:,maxlend] == eos)
    probs = model.predict(x, verbose=0, batch_size=batch_size)
    x_out = x.copy()
    for b in range(batch_size):
        # pick locations we want to flip
        # 0...maxlend-1 are descriptions and should be fixed
        # maxlend is eos and should be fixed
        flips = sorted(random.sample(range(maxlend+1,maxlen), nflips))
        if debug and b < debug:
            print (b)
        for input_idx in flips:
            if x[b,input_idx] == empty or x[b,input_idx] == eos:
                continue
            # convert from input location to label location
            # the output at maxlend (when input is eos) is feed as input at maxlend+1
            label_idx = input_idx - (maxlend+1)
            prob = probs[b, label_idx]
            w = prob.argmax()
            if w == empty:  # replace accidental empty with oov
                w = oov0
            if debug and b < debug:
                print ('%s => %s'%(idx2word[x_out[b,input_idx]],idx2word[w]))
            x_out[b,input_idx] = w
        if debug and b < debug:
            print(1)
    return x_out

In [378]:
def conv_seq_labels(xds, xhs, nflips=None, model=None, debug=False):
    """description and hedlines are converted to padded input vectors. headlines are one-hot to label"""
    batch_size = len(xhs)
    assert len(xds) == batch_size
    x = [vocab_fold(lpadd(xd)+xh) for xd,xh in zip(xds,xhs)]  # the input does not have 2nd eos
    x = sequence.pad_sequences(x, maxlen=maxlen, value=empty, padding='post', truncating='post')
    x = flip_headline(x, nflips=nflips, model=model, debug=debug)
    
    y = np.zeros((batch_size, maxlenh, vocab_size))
    for i, xh in enumerate(xhs):
        xh = vocab_fold(xh) + [eos] + [empty]*maxlenh  # output does have a eos at end
        xh = xh[:maxlenh]
        y[i,:,:] = np_utils.to_categorical(xh, vocab_size)
        
    return x, y

In [379]:
def gen(Xd, Xh, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, seed=seed):
    """yield batches. for training use nb_batches=None
    for validation generate deterministic results repeating every nb_batches
    
    while training it is good idea to flip once in a while the values of the headlines from the
    value taken from Xh to value generated by the model.
    """
    c = nb_batches if nb_batches else 0
    while True:
        xds = []
        xhs = []
        if nb_batches and c >= nb_batches:
            c = 0
        new_seed = random.randint(0, 9223372036854775807)
        random.seed(c+123456789+seed)
        for b in range(batch_size):
            t = random.randint(0,len(Xd)-1)

            xd = Xd[t]
            s = random.randint(min(maxlend,len(xd)), max(maxlend,len(xd)))
            xds.append(xd[:s])
            
            xh = Xh[t]
            s = random.randint(min(maxlenh,len(xh)), max(maxlenh,len(xh)))
            xhs.append(xh[:s])

        # undo the seeding before we yield inorder not to affect the caller
        c+= 1
        random.seed(new_seed)

        yield conv_seq_labels(xds, xhs, nflips=nflips, model=model, debug=debug)

In [380]:
r = next(gen(X_train, Y_train, batch_size=batch_size))
r[0].shape, r[1].shape, len(r)

((20, 50), (20, 25, 40000), 2)

In [381]:
def test_gen(gen, n=5):
    Xtr,Ytr = next(gen)
    for i in range(n):
        assert Xtr[i,maxlend] == eos
        x = Xtr[i,:maxlend]
        y = Xtr[i,maxlend:]
        yy = Ytr[i,:]
        yy = np.where(yy)[1]
        prt('L',yy)
        prt('H',y)
        if maxlend:
            prt('D',x)

In [385]:
test_gen(gen(X_train, Y_train, batch_size=batch_size))

L:
In
Situ
Variability
of
Large
Aggregates
over
a
Fluid
Mud
Bed
Shear
Effects
Settling
Velocities
and
Response
to
Storms
~
_
_
_
_
_
H:
~
In
Situ
Variability
of
Large
Aggregates
over
a
Fluid
Mud
Bed
Shear
Effects
Settling
Velocities
and
Response
to
Storms
_
_
_
_
_
D:
Transects
of
stations
across
Cape
Lookout
Bight
will
be
reoccupied
every
3
months
for
2
years
vertical
profiles
will
be
reoccuppied
every
3
months
for
L:
Preparing
Undergraduate
Engineers
for
Research
in
Biophysical
Systems
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
H:
~
Preparing
Undergraduate
Engineers
for
Research
in
Biophysical
Systems
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
D:
work
and
stimulating
research
projects
have
been
planned
to
instill
an
interest
in
and
enthusiasm
for
future
research
activities
Several
research
projects
applying
biophysical
processes
L:
Flow
Injection
Analyzer
for
in
Situ
Oceanic
Chemical
Analyses
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
H:
~
Flow
Injection
Analyzer
for
in
Situ
Oceanic
Chemical
Analyses
_
_
_
_
_
_
_

In [386]:
history = {}

In [387]:
traingen = gen(X_train, Y_train, batch_size=batch_size, nflips=nflips, model=model)
valgen = gen(X_test, Y_test, batch_size=batch_size)

In [388]:
r = next(traingen)
r[0].shape, r[1].shape, len(r)

((20, 50), (20, 25, 40000), 2)

In [389]:
nflips

10

In [453]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size,
                    input_length=maxlen,
                    W_regularizer=regularizer, weights=[embedding], mask_zero=True,
                    name='embedding_1'))
#model.add(SpatialDropout1D(p_emb))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True,
                W_regularizer=regularizer, U_regularizer=regularizer,
                b_regularizer=regularizer, dropout_W=p_W, dropout_U=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)
    model.add(Dropout(0,name='dropout_%d'%(i+1)))
# if activation_rnn_size:
#     model.add(SimpleContext(name='simplecontext_1'))
model.add(TimeDistributed(Dense(vocab_size,
                                W_regularizer=regularizer, b_regularizer=regularizer,
                                name = 'timedistributed_1')))

model.add(Activation('softmax', name='activation_1'))
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(40000, 100, input_length=50, weights=[array([[-..., mask_zero=True, name="embedding_1", embeddings_regularizer=None)`
  """
/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, name="lstm_1", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  # This is added back by InteractiveShellApp.init_path()
/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, name="lstm_2", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  # This is added back by InteractiveShellApp.init_path()
/Users

In [454]:
# h = model.fit_generator(traingen, samples_per_epoch=nb_train_samples,
#                         nb_epoch=1, validation_data=valgen, nb_val_samples=nb_val_samples
#   )
#h= model.fit_generator(traingen, samples_per_epoch= 20)
h = model.fit_generator(traingen, samples_per_epoch=20,
                        nb_epoch=1, validation_data=valgen, nb_val_samples=nb_val_samples
                           )

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., steps_per_epoch=20, epochs=1, validation_steps=50)`
  


Epoch 1/1


ValueError: Error when checking target: expected activation_1 to have shape (50, 40000) but got array with shape (25, 40000)

In [289]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size,
                    input_length=maxlen,
                    W_regularizer=regularizer, weights=[embedding], mask_zero=True,
                    name='embedding_1'))
#model.add(SpatialDropout1D(p_emb))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True,
                W_regularizer=regularizer, U_regularizer=regularizer,
                b_regularizer=regularizer, dropout_W=p_W, dropout_U=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)
    model.add(Dropout(p_dense,name='dropout_%d'%(i+1)))
    
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(40000, 100, input_length=50, weights=[array([[-..., mask_zero=True, name="embedding_1", embeddings_regularizer=None)`
  """
/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, return_sequences=True, name="lstm_1", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  # This is added back by InteractiveShellApp.init_path()
/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, return_sequences=True, name="lstm_2", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  # This is added back by InteractiveShellApp.init_path()
/Users

In [295]:
len(Y_train[0])

13

In [297]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents
text = texts[0]
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector = vectorizer.transform([text[0]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

NameError: name 'texts' is not defined

In [298]:
keras.utils.to_categorical(X_train[0], num_classes=40000)

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [303]:
keras.utils.to_categorical(X_train[0:3], num_classes=40000)

ValueError: setting an array element with a sequence.

In [319]:
inputX = np.zeros((len(X_train), 40000))
for i in range(len(X_train)):
    arr = np.zeros(40000)
    for j in X_train[0]:
        arr[j] += 1
    inputX[i] = arr

In [321]:
inputY = np.zeros((len(Y_train), 40000))
for i in range(len(Y_train)):
    arr = np.zeros(40000)
    for j in Y_train[0]:
        arr[j] += 1
    inputY[i] = arr

In [331]:
from keras import Model
from keras.layers import Input, Dense
vocab_size = len(X_train)
src_txt_length = 40000
sum_txt_length = 40000
# encoder input model
inputs = Input(shape=(src_txt_length,))
encoder1 = Embedding(vocab_size, 128)(inputs)
encoder2 = LSTM(128)(encoder1)
encoder3 = RepeatVector(sum_txt_length)(encoder2)
# decoder output model
decoder1 = LSTM(128, return_sequences=True)(encoder3)
outputs = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder1)
# tie it together
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [333]:
model = Sequential()
model.add(Embedding(329, 40000, name='embedding_1'))
#model.add(SpatialDropout1D(p_emb))
for i in range(3):
    lstm = LSTM(3, return_sequences=True,
                W_regularizer=regularizer, U_regularizer=regularizer,
                b_regularizer=regularizer, dropout_W=p_W, dropout_U=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)
    model.add(Dropout(0,name='dropout_%d'%(i+1)))
    
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

model.fit(inputX, inputY)

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(3, return_sequences=True, name="lstm_1", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  
/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(3, return_sequences=True, name="lstm_2", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  
/Users/michellezhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(3, return_sequences=True, name="lstm_3", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  


ValueError: Error when checking target: expected dropout_3 to have 3 dimensions, but got array with shape (329, 40000)

In [330]:
inputX.shape

(329, 40000)